In [1]:
pip install qiskit


  Obtaining dependency information for qiskit from https://files.pythonhosted.org/packages/ba/c9/4a35428070cfb6633d72afc077fdbe5adca08a462b401f99647e8a4a4925/qiskit-2.2.3-cp39-abi3-win_amd64.whl.metadata
  Obtaining dependency information for rustworkx>=0.15.0 from https://files.pythonhosted.org/packages/a9/ec/cee878c1879b91ab8dc7d564535d011307839a2fea79d2a650413edf53be/rustworkx-0.17.1-cp39-abi3-win_amd64.whl.metadata
  Obtaining dependency information for stevedore>=3.0.0 from https://files.pythonhosted.org/packages/f4/40/8561ce06dc46fd17242c7724ab25b257a2ac1b35f4ebf551b40ce6105cfa/stevedore-5.6.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/7.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/7.8 MB 991.0 kB/s eta 0:00:08
    --------------------------------------- 0.1/7.8 MB 1.3 MB/s eta 0:00:06
    --------------------------------------- 0.2/7.8 MB 1.3 MB/s eta 0:00:06
    --------------------------------------- 0.2/7.8 MB 1.3 MB/s eta 

In [2]:
pip install qiskit matplotlib


Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install qiskit-aer


  Obtaining dependency information for qiskit-aer from https://files.pythonhosted.org/packages/82/8f/ac0a55a6fb539355e9e629f42f03032f9e93acce2390a87a7ab8c56764f7/qiskit_aer-0.17.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.6 MB 991.0 kB/s eta 0:00:10
   ---------------------------------------- 0.1/9.6 MB 1.3 MB/s eta 0:00:08
    --------------------------------------- 0.2/9.6 MB 1.3 MB/s eta 0:00:08
    --------------------------------------- 0.2/9.6 MB 1.3 MB/s eta 0:00:08
    --------------------------------------- 0.2/9.6 MB 1.3 MB/s eta 0:00:08
    --------------------------------------- 0.2/9.6 MB 1.3 MB/s eta 0:00:08
   - -------------------------------------- 0.4/9.6 MB 1.3 MB/s eta 0:00:07
   - -------------------------------------- 0.4/9.6 MB 1.3 MB/s eta 0:00:07
   - -------------------------------------- 0.4/9.6 MB 1.1 MB/s eta 0:00:09
   -- ------------------------

In [7]:
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit_aer import Aer
from qiskit import transpile



In [13]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np


def build_teleportation_like_circuit(alpha=1/np.sqrt(2), beta=1/np.sqrt(2)):
    """
    Simplified teleportation-style circuit *without* dynamic classical control.

    q0: state to teleport (Alice)
    q1: Alice's half of Bell pair
    q2: Bob's half of Bell pair

    c0, c1: Alice's measurement bits
    cb: Bob's measurement bit
    """
    # Normalize state
    norm = np.sqrt(abs(alpha)**2 + abs(beta)**2)
    alpha /= norm
    beta /= norm

    q = QuantumRegister(3, "q")
    c = ClassicalRegister(3, "c")  # we'll store [Alice0, Alice1, Bob]
    qc = QuantumCircuit(q, c)

    # 1. Prepare |ψ> on qubit 0
    qc.initialize([alpha, beta], q[0])

    # 2. Create Bell pair between q1 and q2
    qc.h(q[1])
    qc.cx(q[1], q[2])

    # 3. Bell measurement on q0 and q1
    qc.cx(q[0], q[1])
    qc.h(q[0])

    # Measure Alice's two qubits
    qc.measure(q[0], c[0])  # Alice bit 0
    qc.measure(q[1], c[1])  # Alice bit 1

    # 4. Measure Bob's qubit (without doing classical corrections)
    qc.measure(q[2], c[2])  # Bob bit

    return qc


def run_teleportation_like(alpha=1/np.sqrt(2), beta=1/np.sqrt(2), shots=1024):
    qc = build_teleportation_like_circuit(alpha, beta)
    print("Simplified teleportation-style circuit (no dynamic control):")
    print(qc.draw())

    backend = Aer.get_backend("qasm_simulator")
    qc_t = transpile(qc, backend)
    job = backend.run(qc_t, shots=shots)
    result = job.result()
    counts = result.get_counts()

    print("\nMeasurement counts (c2 c1 c0) = (Bob, Alice1, Alice0) in Qiskit's bitstring order:")
    print(counts)

    plot_histogram(counts)
    plt.show()


if __name__ == "__main__":
    print("Input state |0>")
    run_teleportation_like(alpha=1, beta=0)

    print("\nInput state |1>")
    run_teleportation_like(alpha=0, beta=1)

    print("\nInput state |+> = (|0> + |1>)/sqrt(2)")
    run_teleportation_like(alpha=1/np.sqrt(2), beta=1/np.sqrt(2))



Input state |0>
Simplified teleportation-style circuit (no dynamic control):
     ┌─────────────────┐          ┌───┐┌─┐
q_0: ┤ Initialize(1,0) ├───────■──┤ H ├┤M├
     └──────┬───┬──────┘     ┌─┴─┐└┬─┬┘└╥┘
q_1: ───────┤ H ├─────────■──┤ X ├─┤M├──╫─
            └───┘       ┌─┴─┐└┬─┬┘ └╥┘  ║ 
q_2: ───────────────────┤ X ├─┤M├───╫───╫─
                        └───┘ └╥┘   ║   ║ 
c: 3/══════════════════════════╩════╩═══╩═
                               2    1   0 

Measurement counts (c2 c1 c0) = (Bob, Alice1, Alice0) in Qiskit's bitstring order:
{'110': 268, '001': 269, '111': 257, '000': 230}

Input state |1>
Simplified teleportation-style circuit (no dynamic control):
     ┌─────────────────┐          ┌───┐┌─┐
q_0: ┤ Initialize(0,1) ├───────■──┤ H ├┤M├
     └──────┬───┬──────┘     ┌─┴─┐└┬─┬┘└╥┘
q_1: ───────┤ H ├─────────■──┤ X ├─┤M├──╫─
            └───┘       ┌─┴─┐└┬─┬┘ └╥┘  ║ 
q_2: ───────────────────┤ X ├─┤M├───╫───╫─
                        └───┘ └╥┘   ║   ║ 
c: 3/═══════════════════